In [1]:
with open('data/makemore/names.txt', mode='rt', encoding="utf-8") as file:
    names = file.read().splitlines()

In [2]:
N_block = 3

X_lst = []
Y_lst = []

for name in names:
    chrs = list(name) + ['.']
    gram = ['.'] * N_block
    for ch in chrs:
        X_lst.append(gram)
        Y_lst.append(ch)
        gram = gram[1:] + [ch]

In [3]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


N_chars = 27
N_X_tokens = 3
N_Y_tokens = 1
N_embedding_dim = 2

X_tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
X_tokenizer.fit_on_texts(X_lst)
X_token_lst = X_tokenizer.texts_to_sequences(X_lst)
X_index_dct = X_tokenizer.word_index
X = pad_sequences(X_token_lst, maxlen=N_X_tokens) - 1

Y_tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
Y_tokenizer.fit_on_texts(Y_lst)
Y_token_lst = Y_tokenizer.texts_to_sequences(Y_lst)
Y_index_dct = Y_tokenizer.word_index
Y_word_dct = {index: char for char, index in Y_index_dct.items()}
Y_seq = pad_sequences(Y_token_lst, maxlen=N_Y_tokens) - 1
Y = to_categorical(Y_seq)

In [4]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding, Softmax

model = Sequential()
model.add(Embedding(N_chars, N_embedding_dim, input_length=N_X_tokens))
model.add(Flatten())
model.add(Dense(N_chars, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

model.summary()

history = model.fit(X, Y, epochs=6)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 2)              54        
                                                                 
 flatten (Flatten)           (None, 6)                 0         
                                                                 
 dense (Dense)               (None, 27)                189       
                                                                 
Total params: 243 (972.00 Byte)
Trainable params: 243 (972.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/6


7130/7130 [==============================] - 13s 2ms/step - loss: 2.6069 - acc: 0.2318
Epoch 2/6
7130/7130 [==============================] - 9s 1ms/step - loss: 2.4893 - acc: 0.2452
Epoch 3/6
7130/7130 [==============================] - 9s 1ms/step - loss: 2.4820 - a

In [18]:
import numpy as np


N_pred_words = 1

stop_idx = Y_index_dct['.']
for _ in range(N_pred_words):
    tokens = [stop_idx] * N_X_tokens
    word = ''
    while True:
        print(tokens)
        p = np.squeeze(model.predict([tokens]))
        idx = int(np.random.choice(np.arange(N_chars), p=p)) + 1
        # print(idx)
        tokens = tokens[1:] + [idx]
        # tokens = [1] * N_X_tokens
        word += Y_word_dct[idx]
        if idx == stop_idx:
            break
    print(word)

[2, 2, 2]
1/1 [==============================] - 0s 51ms/step
[2, 2, 4]
1/1 [==============================] - 0s 46ms/step
n.
